# Customer Churn Telecom

Customer Churn (ou Rotatividade de Clientes, em uma tradução livre) refere-se a uma decisão tomada pelo cliente sobre o término do relacionamento comercial. Refere-se também à perda de clientes. A fidelidade do cliente e a rotatividade de clientes sempre somam 100%. Se uma empresa tem uma taxa de fidelidade de 60%, então a taxa de perda de clientes é de 40%. De acordo com a
regra de lucratividade do cliente 80/20, 20% dos clientes estão gerando 80% da receita. Portanto, é muito importante prever os usuários que provavelmente abandonarão o relacionamento comercial e os fatores que afetam as decisões do cliente.

Neste projeto, o objetivo é prever o Customer Churn em uma Operadora de Telecom.

## Pré-Processamento de Dados

Antes da etapa de modelagem preditiva, é preciso pré-processar os dados. O pré-processamento é necessário por diversos motivos, entre eles:

* **Lidar com valores _missing_**: Valores _missing_ ou "faltantes" têm impacto significativo na capacidade preditiva dos modelos e por isso devem ser eliminados removendo os registros ou utilizando técnicas de imputação.
* **Diferença de escala nos dados**: Em muitos modelos de Machine Learning, a diferença na escala dos dados pode levar ao treinamento incorreto do modelo por conta de vieses. Logo os dados devem ser padronizados ou normalizados sempre que possível.
* **Engenharia de atributos**: Os dados podem ser combinados ou transformados de modo a construir atributos que possam conter informações mais relevantes para a previsão do modelo.
* **Encoding de variáveis categóricas**: Modelos de Machine Learning são essencialmente matemática e estatística, e portanto não capazes de processar dados em forma literal ou de string. Portanto as variáveis categóricas devem ser codificadas como números.
* **Remoção de _outliers_**: _Outliers_, ou anomalias, são dados que estão muito distantes da média e que podem impactar significativamente a performance dos modelos preditivos.

Portanto, o Pré-Processamento é uma etapa crítica em um processo de Machine Learning para garantir que o modelo preditivo seja treinado com dados de alta qualidade e possua boa performance e previsões precisas.

In [1]:
# Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
df = pd.read_csv("projeto4_telecom_treino.csv", index_col=0)

In [3]:
df_copy = df.copy()

## 1- Lidando com valores missing

É necessário verificar se há valores missing nos dados.

In [4]:
df.isna().sum()

state                            0
account_length                   0
area_code                        0
international_plan               0
voice_mail_plan                  0
number_vmail_messages            0
total_day_minutes                0
total_day_calls                  0
total_day_charge                 0
total_eve_minutes                0
total_eve_calls                  0
total_eve_charge                 0
total_night_minutes              0
total_night_calls                0
total_night_charge               0
total_intl_minutes               0
total_intl_calls                 0
total_intl_charge                0
number_customer_service_calls    0
churn                            0
dtype: int64

Não há valores missing nos dados, pode-se seguir para a próxima etapa.

## 2- Padronizando os dados

Como os dados possuem muitos outliers, a padronização será usada como tratamento para variáveis numéricas.

In [5]:
# Variáveis numéricas
numeric_vars = [var for var in df.columns if df[var].dtype != 'object']
numeric_vars

['account_length',
 'number_vmail_messages',
 'total_day_minutes',
 'total_day_calls',
 'total_day_charge',
 'total_eve_minutes',
 'total_eve_calls',
 'total_eve_charge',
 'total_night_minutes',
 'total_night_calls',
 'total_night_charge',
 'total_intl_minutes',
 'total_intl_calls',
 'total_intl_charge',
 'number_customer_service_calls']

In [6]:
scaler = StandardScaler()
df[numeric_vars] = scaler.fit_transform(df[numeric_vars])

## 3- Encoding de variáveis categóricas

As variáveis categóricas deste dataset são o estado, o código de área do estado, a indicação se o cliente possui plano internacional ou voice-mail e o churn. Não será feito encoding do código de área pois essa variável é redundante com o estado.

In [7]:
## Variáveis categóricas
categorical_vars = [col for col in df.columns if col not in numeric_vars]
categorical_vars

['state', 'area_code', 'international_plan', 'voice_mail_plan', 'churn']

In [8]:
df = df.drop(columns = 'area_code')

In [9]:
# State
lab_encoder = LabelEncoder()
df['state'] = lab_encoder.fit_transform(df['state'])

In [10]:
# Planos e churn
df['international_plan'] = np.where(df['international_plan'] == 'yes', 1, 0)
df['voice_mail_plan'] = np.where(df['voice_mail_plan'] == 'yes', 1, 0)
df['churn'] = np.where(df['churn'] == 'yes', 1, 0)

## 4- Engenharia de atributos

Serão criadas duas novas variáveis:

* special_plan: Variável categórica que indica se o cliente tem plano internacional ou plano voice-mail. Essencialmente um "OR" lógico.
* old_account: Variável categórica que indica se o cliente tem a conta há mais de 100 dias ou não. (binária)


In [11]:
df['special_plan'] = np.where((df['international_plan'] == 'yes') | (df['voice_mail_plan'] == 'yes'), 1, 0)
df['special_plan'].value_counts()

0    3333
Name: special_plan, dtype: int64

In [12]:
# Usando df_copy pois contém o valor original de account_length
df['old_account'] = np.where(df_copy['account_length'] > 100, 1, 0)
df['old_account'].value_counts()

1    1669
0    1664
Name: old_account, dtype: int64

## 5- Remoção de outliers

Primeiro é necessário verificar quantos outliers existem em cada variável.

In [13]:
# Número de outliers por variável
abs(df[numeric_vars] > 3).sum()

account_length                    7
number_vmail_messages             3
total_day_minutes                 3
total_day_calls                   2
total_day_charge                  3
total_eve_minutes                 3
total_eve_calls                   3
total_eve_charge                  3
total_night_minutes               7
total_night_calls                 3
total_night_charge                7
total_intl_minutes                2
total_intl_calls                 50
total_intl_charge                 2
number_customer_service_calls    35
dtype: int64

In [14]:
# Filtrando outliers
df = df[abs(df[numeric_vars] <= 3).all(axis = 1)]

In [15]:
# Número de outliers por variável
abs(df[numeric_vars] > 3).sum()

account_length                   0
number_vmail_messages            0
total_day_minutes                0
total_day_calls                  0
total_day_charge                 0
total_eve_minutes                0
total_eve_calls                  0
total_eve_charge                 0
total_night_minutes              0
total_night_calls                0
total_night_charge               0
total_intl_minutes               0
total_intl_calls                 0
total_intl_charge                0
number_customer_service_calls    0
dtype: int64

In [16]:
# Tamanho do dataframe após filtrar outliers
len(df)

3215

In [17]:
print(f'O novo dataset tem {len(df)/len(df_copy)*100:.2f}% dos dados do anterior.')

O novo dataset tem 96.46% dos dados do anterior.


Os dados são suficientes para o treinamento do modelo.

## Pré-Processando dados de teste

Para avaliar do modelo será necessário aplicar todas as transformações que foram aplicadas aos dados de treino também aos dados de teste.

In [18]:
# Carregando dataset
df_test = pd.read_csv("projeto4_telecom_teste.csv", index_col=0)
df_test

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
1,HI,101,area_code_510,no,no,0,70.9,123,12.05,211.9,73,18.01,236.0,73,10.62,10.6,3,2.86,3,no
2,MT,137,area_code_510,no,no,0,223.6,86,38.01,244.8,139,20.81,94.2,81,4.24,9.5,7,2.57,0,no
3,OH,103,area_code_408,no,yes,29,294.7,95,50.10,237.3,105,20.17,300.3,127,13.51,13.7,6,3.70,1,no
4,NM,99,area_code_415,no,no,0,216.8,123,36.86,126.4,88,10.74,220.6,82,9.93,15.7,2,4.24,1,no
5,SC,108,area_code_415,no,no,0,197.4,78,33.56,124.0,101,10.54,204.5,107,9.20,7.7,4,2.08,2,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1663,HI,50,area_code_408,no,yes,40,235.7,127,40.07,223.0,126,18.96,297.5,116,13.39,9.9,5,2.67,2,no
1664,WV,152,area_code_415,no,no,0,184.2,90,31.31,256.8,73,21.83,213.6,113,9.61,14.7,2,3.97,3,yes
1665,DC,61,area_code_415,no,no,0,140.6,89,23.90,172.8,128,14.69,212.4,97,9.56,13.6,4,3.67,1,no
1666,DC,109,area_code_510,no,no,0,188.8,67,32.10,171.7,92,14.59,224.4,89,10.10,8.5,6,2.30,0,no


In [19]:
# Cópia dos dados de teste
df_test_copy = df_test.copy()

# Valores missing
df_test.dropna()

# Padronização
df_test[numeric_vars] = scaler.transform(df_test[numeric_vars])

# Encoding das variáveis categóricas
df_test = df_test.drop(columns='area_code')
df_test['state'] = lab_encoder.transform(df_test['state'])
df_test['international_plan'] = np.where(df_test['international_plan'] == 'yes', 1, 0)
df_test['voice_mail_plan'] = np.where(df_test['voice_mail_plan'] == 'yes', 1, 0)
df_test['churn'] = np.where(df_test['churn'] == 'yes', 1, 0)

# Engenharia de atributos
df_test['special_plan'] = np.where((df_test['international_plan'] == 'yes') | (df_test['voice_mail_plan'] == 'yes'), 1, 0)
df_test['old_account'] = np.where(df_test_copy['account_length'] > 100, 1, 0)

# A remoção de outliers não é necessária para dados de teste, apenas para treinamento do modelo

## Salvando os dados pré-processados

Após todo o pré-processamento, os dados serão salvos para prosseguir para modelagem.

In [20]:
df.to_csv("dados_treino_preproc.csv", index=False)
df_test.to_csv("dados_teste_preproc.csv", index=False)